# 数据展示与预处理

In [1]:
import pandas as pd
import numpy as np
import regex as re 
import numpy as np
import os

## 数据集的基本信息如下：
有1953条健康信息断言，不包含index有8列
- 第1列“rumor”为信息的标签，二分类，0表示非谣言，1表示谣言
- 第2列“text”为人工标注的健康信息断言，仅清洗特殊字符、使用PunctualModel补充标点
- 第3列“seg_text”为使用哈工大停用词表去除停用词后的词表
- 第4列“cos_matrix”为余弦相似度矩阵，按照行优先存储，每一个行向量表示该文档与其他所有文档的余弦相似度
- 第5列“length”为“text”列的文档的长度
- 第6列“sentiment”为情感极性得分，使用SnowNLP计算得到
- 第7列“pca_topic_vectors”为使用sklearn中的TurncatedPCA方法计算得到的主题得分，我选择了100个维度
- 第8列“lida_topic_vectors”使用lida得到，目前暂时是32维的主题得分

In [2]:
data = pd.read_csv('data.csv',index_col=0)
data.columns.values

array(['rumor', 'text', 'seg_text', 'cos_sim', 'length', 'sentiments',
       'pca_topic_vectors', 'ldia_topic_vectors'], dtype=object)

In [3]:
print("谣言数量："+str(len(data[data['rumor']==1])))
print("非谣言数量："+str(len(data[data['rumor']==0])))

谣言数量：913
非谣言数量：1871


In [4]:
print("全部平均长度："+str(sum([len(d) for d in data.text])/len(data)))
print("谣言平均长度："+str(sum([len(d) for d in data[data['rumor']==1].text])/len(data[data['rumor']==1])))
print("非谣言平均长度："+str(sum([len(d) for d in data[data['rumor']==0].text])/len(data[data['rumor']==0])))

全部平均长度：61.10596264367816
谣言平均长度：16.00985761226725
非谣言平均长度：83.11170497060395


## 首先制作一个符合Chinese-Text- Classification-Pytorch的数据集

In [5]:

texts = np.array([data.index,data['seg_text'],data['rumor']]).T
np.random.seed(2)
np.take(texts,np.random.permutation(texts.shape[0]),axis=0,out=texts)
train,test,dev = np.split(texts,[int(texts.shape[0]*0.6),int(texts.shape[0]*0.8)],axis=0)
train = train.T
test = test.T
dev = dev.T
print("训练集中的谣言数量:"+str(sum(train[2])))
print("测试集中的谣言数量:"+str(sum(test[2])))
print("验证集中的谣言数量:"+str(sum(dev[2])))
with open('./pygp_data/data/train.txt','w') as f:
    for i in range(train.shape[1]):
        f.write(str(train[0][i])+'\t'+str(train[1][i])+'\t'+str(train[2][i])+'\n')
with open('./pygp_data/data/test.txt','w') as f:
    for i in range(test.shape[1]):
        f.write(str(test[0][i])+'\t'+str(test[1][i])+'\t'+str(test[2][i])+'\n')
with open('./pygp_data/data/dev.txt','w') as f:
    for i in range(dev.shape[1]):
        f.write(str(dev[0][i])+'\t'+str(dev[1][i])+'\t'+str(dev[2][i])+'\n')


训练集中的谣言数量:536
测试集中的谣言数量:196
验证集中的谣言数量:181
